# Find identity term matches in data

See process_data.py

In [1]:
# Load incels data
import pandas as pd
import csv

path = '../../data/incels/all_comments.csv'
# data = pd.read_csv(path, engine='python', on_bad_lines=lambda x: print(x))
data = pd.read_csv(path, engine='python', on_bad_lines=lambda row: row[:-2].append(' '.join(row[-2:]))) # combine last 2 elements in a line mentioning Gulag
data['parsed_date'] = pd.to_datetime(data.date, errors='coerce') # "yesterday" etc not handled
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248230 entries, 0 to 6248229
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   type         object        
 1   forum        object        
 2   thread       object        
 3   username     object        
 4   date         object        
 5   content      object        
 6   parsed_date  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 333.7+ MB


## NetMapper identity term lists

### generic_agents-identity 2021 file

In [2]:
# Load identity terms
import pandas as pd

nm_identities_path = '../resources/generic_agents-identity_v15_2021_10_15.xlsx'
multi_identities = pd.read_excel(nm_identities_path)
multi_identities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19887 entries, 0 to 19886
Columns: 156 entries, Akan to Category 4
dtypes: float64(56), int64(1), object(99)
memory usage: 23.7+ MB


In [3]:
# Filter to English, remove duplicates
cols = multi_identities.columns.tolist()
en_identities = multi_identities[cols[cols.index('English'):]]
en_identities['term'] = en_identities['English'].str.lower()
en_identities.drop_duplicates(subset='term', inplace=True)
en_identities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19035 entries, 0 to 19886
Data columns (total 44 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   English          19035 non-null  object 
 1   conceptTo        19035 non-null  object 
 2   metaOntology     19035 non-null  object 
 3   nodeType         0 non-null      float64
 4   Category 1       9703 non-null   object 
 5   Category 2       911 non-null    object 
 6   Category 3       18929 non-null  object 
 7   Country          4712 non-null   object 
 8   First Name       0 non-null      float64
 9   Last Name        0 non-null      float64
 10  Gender           759 non-null    object 
 11  Suffix           0 non-null      float64
 12  Language         0 non-null      float64
 13  Acronym          336 non-null    object 
 14  Valence          19035 non-null  object 
 15  Evaluation       19035 non-null  int64  
 16  Potency          10 non-null     float64
 17  Activity    

/tmp/ipykernel_1221270/3338936415.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_identities['term'] = en_identities['English'].str.lower()
/tmp/ipykernel_1221270/3338936415.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_identities.drop_duplicates(subset='term', inplace=True)


In [3]:
en_identities['Gender'].value_counts()

female     364
male       341
neutral     49
yes          5
Name: Gender, dtype: int64

In [4]:
en_identities['Political Role'].value_counts()

Minister            391
Governor             63
President            58
Diplomat             51
Prime Minister       31
MP                   31
Vice President       26
Mayor                24
Leader               21
Attorney_General     16
Prince               15
MFA                  12
Secretary             9
Vice-President        7
Queen                 7
Princess              6
Vice Premier          4
Congressman           4
King                  3
Royalty               2
Lt_Governor           2
PM                    2
Senator               1
Name: Political Role, dtype: int64

In [7]:
en_identities['Religious Role '].value_counts()

Sheik         21
bishop        16
Archbishop    13
Priest        12
Pope           7
Rabbi          5
Sheikh         2
archbishop     1
Name: Religious Role , dtype: int64

In [9]:
pd.set_option('display.max_rows', None)
en_identities['Category 1'].value_counts()

nationality                4013
Celebrity_Politician        989
ethnic_group                804
military                    694
abusive                     414
scientist                   283
law_enforcement             231
criminal                    198
medical                     182
legal                       169
religious                   146
athlete                     141
religious_leader            135
manager                     105
journalism                  103
student                      84
aristocrat                   76
musician                     73
family_adult_male            72
family_adult_female          71
writer                       61
location_identity            60
stopword                     55
child                        53
kin                          50
artist                       48
entertainer                  41
family_child_female          38
leader                       35
family_child_male            32
sportsperson                 30
Calebrit

In [10]:
en_identities['Category 2'].value_counts()

Native American    644
family_female      118
family_male        109
family-child        17
attacker            10
family_adult         9
defender             2
cyber_actor          2
Name: Category 2, dtype: int64

In [11]:
en_identities['Category 3'].value_counts()

identity    18929
Name: Category 3, dtype: int64

#### Check if terms are in the list

In [25]:
# From WebSci 2020 paper, LGBTQ+ movement dissertation chapter
terms = ['gurl', 'lgbt', 'lgbtq', 'genderfluid', 'gender-fluid', 'gender fluid', 'straight', 'cisgender', 'cis', 
         'asexual', 'aro-ace', 'aro ace', 'aro/ace', 'mtf', 'ftm', 
         'neurodivergent', 'neurodiversity', 'autism', 'autist', 'autistic','mental health', 'adhd', 'dyslexia', 'dyslexic', 
         "asperger's", "aspergers", 'asperger', 'savant', 'intellectual disability', 'bipolar', 'schizophrenia', 'ocd', 'antisocial', 
         'obsessive-compulsive', 'obsessive compulsive', 'disability', 'disabilities', 'disabled', 'anxiety disorder', 'retarded', 'depression',
        'depressive', 'sandnigger', 'sandniggers']
for term in terms:
    print(term, term in en_identities.term.tolist())

gurl False
lgbt False
lgbtq False
genderfluid False
gender-fluid False
gender fluid False
straight False
cisgender False
cis False
asexual False
aro-ace False
aro ace False
aro/ace False
mtf False
ftm False
neurodivergent False
neurodiversity False
autism False
autist False
autistic False
mental health False
adhd False
dyslexia False
dyslexic False
asperger's False
aspergers False
asperger False
savant True
intellectual disability False
bipolar False
schizophrenia False
ocd False
antisocial False
obsessive-compulsive False
obsessive compulsive False
disability False
disabilities False
disabled False
anxiety disorder False
retarded False
depression False
depressive False
sandnigger True
sandniggers False


In [17]:
# From CoNLL 2022 paper
terms = ['black folks', 'caucasian', 'caucasians', 'heterosexual', 'heterosexuals', 'homosexuals', 'homosexual', 'buddhist', 'indigenous',
        'native american', 'native americans', 'young adults', 'middle aged', 'seniors', 'old people', 'elderly', 'immigrants', 'refugees', 'immigrant', 'refugee',
        'migrant', 'lesbian', 'lesbians', 'middle eastern', 'illegal aliens', 'arabian', 'arabians']
for term in terms:
    print(term, term in en_identities.term.tolist())

black folks False
caucasian True
caucasians False
heterosexual False
heterosexuals False
homosexuals True
homosexual True
buddhist True
indigenous False
native american True
native americans True
young adults False
middle aged False
seniors True
old people True
elderly True
immigrants True
refugees True
immigrant True
refugee True
migrant True
lesbian True
lesbians True
middle eastern False
illegal aliens False
arabian True
arabians True


In [ ]:
[identity for identity in en_identities.term if 'black' in identity]

In [ ]:
[identity for identity in en_identities.term if 'white' in identity]

In [18]:
# Separate out stopwords
stops = en_identities[en_identities['stop word']==1]
exclude_list = ['don', 'other', 'others', 'friend', 'friends', 'people', 'who', 'asshole', 'dick',
               'character', 'person', 'people', 'majority', 'bot', 'everyone', 'everyone here',
                'officially', 'tech', 'individual', 'worker', 'workers', 'giant', 'human', 'humans', 'ass',
                'nobody', 'brother', 'sister', 'mother', "mother's", 'father', 'daughter', 'son', 'mom', 'wife', 'wives', 'husband', 'husbands', 'cousin', 'cousins',
                'they all', 'count', 'god', 'general', 'user', 'users', 'member', 'members', 'english', 'finish', 'slayer', 'speaker',
                'misogynist', 'king', 'queen', 'rn', 'fellow', 'buddy', 'enemies', 'corpse', 'revolutionary', 'gymnast', 'messiah', 'jesus', 'embryo',
                'dr', 'doctor', 'dahmer', 'characters', 'cheat', 'sexist', 'professional', 'client', 'mate', 'dad', 'customers', 'assholes', 'whose',
                'mama', 'co-workers', 'employees', 'uncle', 'hermit', 'ogre', 'potter', 'phantom', 'dwellers', 'saviour', 'prophet', 'morons', 'guide',
                'majors', 'partners', 'villain', 'agent', 'model', 'juggernaut', 'ego', 'avatar', 'player', 'dragon', 'pm', 'winner', 'winners', 'surrogate', 'nudes',
               ]
exclude = en_identities[en_identities['term'].isin(exclude_list)]
identities = en_identities[(en_identities['stop word']!=1) & (~en_identities['term'].isin(exclude_list))]

# Search for identity matches
import re

pats = [re.compile(r'\b{}\b'.format(re.escape(term))) for term in identities['term']]
print(len(pats))

def contains_match(text):
    """ Return matched patterns present in text """ 
    return [re.search(p, str(text).lower()).group() for p in pats if re.search(p, str(text).lower()) is not None]

from collections import Counter

sample = data.sample(1000)

18837


#### Check if terms are in incels data vocab

In [4]:
%%timeit -n 1 -r 1
# Filter to terms present at least once in the combined text (make this as fast as possible)
# vocab = set(data.content.astype('str').str.lower().str.split().sum()) # doesn't tokenize (remove punct) but whatever
vocab = set()
data.content.astype('str').str.lower().str.split().apply(vocab.update)
len(vocab)

28.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [4]:
# Try unigrams and bigrams with sklearn
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,2), min_df=1)
vectorizer.fit(data.content.astype('str'))

CountVectorizer(ngram_range=(1, 2))

In [6]:
len(vectorizer.vocabulary_)

13283289

In [8]:
%%timeit -n 1 -r 1

# Try unigrams and bigrams with sklearn
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,2), min_df=1)
vectorizer.fit(data.content.astype('str'))

4min 38s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
# Never finished--waaaay longer than 5 mins
import itertools

vocab = set()
splits = data.content.astype('str').str.lower().str.split().apply(lambda x: vocab.update(itertools.permutations(x, 2)))
len(vocab)

In [19]:
from tqdm.notebook import tqdm

present_terms = [term for term in identities['term'] if term in vocab]
len(present_terms)

5564

In [21]:
pats_combo = re.compile(r'|'.join([(r'\b{}\b'.format(re.escape(term.lower()))) for term in present_terms]))
# pats_combo

def contains_match(text):
    """ Return matched patterns present in text """ 
    return re.findall(pats_combo, str(text).lower())

In [21]:
%%timeit -n 1 -r 1
# With separate re.search

sample['nm_identity_matches'] = sample.content.map(contains_match)
# sample[sample.nm_identity_matches.str.len() > 0]

58.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
# %%timeit -n 1 -r 3
# With combined re

sample['nm_identity_matches'] = sample.content.map(contains_match)
# sample[sample.nm_identity_matches.str.len() > 0]

In [ ]:
identity_ctr = Counter(sample.nm_identity_matches.sum())
identity_ctr.most_common()

### Identities_tags 2022 (from Josh)

In [36]:
# Load identity terms
import pandas as pd

nm_identities_path = '../resources/identities_tags_v2_2022_5_26.xslx'
nm_identities = pd.read_excel(nm_identities_path)
nm_identities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7166 entries, 0 to 7165
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IDENTITY          7166 non-null   object 
 1   FAMILY            7166 non-null   int64  
 2   POLITICAL         7165 non-null   object 
 3   GENDER            7165 non-null   float64
 4   RELIGION          7166 non-null   int64  
 5   RACE/NATIONALITY  7166 non-null   int64  
 6   JOB               7166 non-null   int64  
 7   OTHER             7166 non-null   int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 448.0+ KB


In [56]:
new_identities = nm_identities.IDENTITY.str.replace('_', ' ').str.lower()
len(set(new_identities).intersection(en_identities.term))

6767

In [42]:
# Filter to terms present at least once in the combined text (make this as fast as possible)
vocab = set()
data.content.astype('str').str.lower().str.split().apply(vocab.update)
len(vocab)

1782573

In [57]:
from tqdm.notebook import tqdm

old_present_terms = [term for term in en_identities['term'] if term in vocab]
print(len(old_present_terms))
new_present_terms = [term for term in new_identities if term in vocab]
print(len(new_present_terms))

5755
2587


In [ ]:
# Look at terms in the old list that aren't present in the new one
both = set(old_present_terms).intersection(set(new_present_terms))
only_old = set(old_present_terms) - both
print(len(only_old))
only_old

In [ ]:
only_new = set(new_present_terms) - both
print(len(only_new))
only_new

#### Get annotations for identity categories
Save out to identity_categories.json

In [61]:
filtered = nm_identities[nm_identities.IDENTITY.str.lower().isin(new_present_terms)]
len(filtered)

2587

In [52]:
filtered.columns

Index(['IDENTITY', 'FAMILY', 'POLITICAL', 'GENDER', 'RELIGION',
       'RACE/NATIONALITY', 'JOB', 'OTHER'],
      dtype='object')

In [85]:
political_present = filtered.loc[nm_identities.POLITICAL == 1, 'IDENTITY'].str.lower()
annotations = {term: 'politics' for term in political_present}
len(annotations)

98

In [86]:
gender_present = filtered.loc[nm_identities.GENDER == 1, 'IDENTITY'].str.lower()
annotations.update({term: 'gender/sexuality' for term in gender_present})
len(annotations)

268

In [87]:
religion_present = filtered.loc[nm_identities.RELIGION == 1, 'IDENTITY'].str.lower()
annotations.update({term: 'religion' for term in religion_present})
len(annotations)

366

In [88]:
race_nationality_present = filtered.loc[nm_identities['RACE/NATIONALITY'] == 1, 'IDENTITY'].str.lower()
annotations.update({term: 'race/ethnicity' for term in race_nationality_present}) # shouldn't really include nationalities
len(annotations)

708

In [95]:
identity_cats.update(annotations)
len(identity_cats)

980

In [97]:
# Save out to identity_categories.json
identity_cats_fpath = '../resources/identity_categories.json'
with open(identity_cats_fpath, 'w') as f:
    json.dump(identity_cats, f, indent=4)

# Visualize identity term matches

In [1]:
# Load incels data with identity term matches extracted
import pandas as pd

path = '../../data/incels/processed_comments.pkl'
incels_data = pd.read_pickle(path)
incels_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248230 entries, 0 to 6248229
Data columns (total 11 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   type                              object        
 1   forum                             object        
 2   thread                            object        
 3   username                          object        
 4   date                              object        
 5   content                           object        
 6   parsed_date                       datetime64[ns]
 7   content_orig                      object        
 8   netmapper_identity_matches        object        
 9   netmapper_identity_matches_spans  object        
 10  actions_attributes                object        
dtypes: datetime64[ns](1), object(10)
memory usage: 524.4+ MB


In [2]:
incels_data['parsed_date'] = pd.to_datetime(incels_data.parsed_date, utc=True)
incels_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248230 entries, 0 to 6248229
Data columns (total 11 columns):
 #   Column                            Dtype              
---  ------                            -----              
 0   type                              object             
 1   forum                             object             
 2   thread                            object             
 3   username                          object             
 4   date                              object             
 5   content                           object             
 6   parsed_date                       datetime64[ns, UTC]
 7   content_orig                      object             
 8   netmapper_identity_matches        object             
 9   netmapper_identity_matches_spans  object             
 10  actions_attributes                object             
dtypes: datetime64[ns, UTC](1), object(10)
memory usage: 524.4+ MB


In [16]:
print(incels_data.parsed_date.min())
print(incels_data.parsed_date.max())

2017-11-07 00:00:00
2021-04-07 00:00:00


In [13]:
# Load ws data with identity term matches extracted
import pandas as pd

path = '../data/white_supremacist_identities.pkl'
ws_data = pd.read_pickle(path)
ws_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4370588 entries, qian2018_0 to rieger2021_white_supremacist_4447
Data columns (total 7 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   text                        object             
 1   word_count                  int64              
 2   dataset                     object             
 3   source                      object             
 4   domain                      object             
 5   timestamp                   datetime64[ns, UTC]
 6   netmapper_identity_matches  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 266.8+ MB


In [17]:
ws_data.domain.value_counts()

forum                 4196731
tweet                   89444
long-form               43016
chat                    39578
offline_propaganda       1819
Name: domain, dtype: int64

In [35]:
# Filter to match incels time period and domain
# from pandas.tseries.offsets import *
from datetime import datetime, timezone

custom_start = datetime(2015,1,1, tzinfo=timezone.utc)
# selected_ws = ws_data[(ws_data.timestamp >= (incels_data.parsed_date.min()) - DateOffset(years=1)) &
selected_ws = ws_data[(ws_data.timestamp >= custom_start) &
                      (ws_data.timestamp <= incels_data.parsed_date.max()) & 
                      (ws_data.domain == 'forum')]
# selected_ws = selected_ws.groupby('source').sample(50000)
selected_ws.info()
print(selected_ws.timestamp.min())
print(selected_ws.timestamp.max())

print(selected_ws.source.value_counts())
print(selected_ws.source.value_counts(normalize=True))

<class 'pandas.core.frame.DataFrame'>
Index: 3410623 entries, ironmarch_0 to papasavva2020_3068515
Data columns (total 7 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   text                        object             
 1   word_count                  int64              
 2   dataset                     object             
 3   source                      object             
 4   domain                      object             
 5   timestamp                   datetime64[ns, UTC]
 6   netmapper_identity_matches  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 208.2+ MB
2015-01-01 00:02:00+00:00
2019-11-01 09:25:21+00:00
4chan         3264407
ironmarch       93377
stormfront      52839
Name: source, dtype: int64
4chan         0.957129
ironmarch     0.027378
stormfront    0.015492
Name: source, dtype: float64


## Total matches

In [ ]:
incels_exp = incels_data.explode('netmapper_identity_matches')
# freq = pd.DataFrame(exploded.netmapper_identity_matches.value_counts(), columns=['term', 'count'])

freq = incels_exp.netmapper_identity_matches.value_counts().to_frame(name='count').reset_index(names='identity_term')
freq.head(20)

import plotly.express as px
pd.options.plotting.backend = 'plotly'

fig = px.bar(freq.head(50), x='identity_term', y='count')
fig.update_xaxes(tickangle=45)

### White supremacist

In [44]:
# ws_exp = ws_data.explode('netmapper_identity_matches')
ws_exp = selected_ws.explode('netmapper_identity_matches')

### Generic-only list vs novel terms

In [4]:
# Load generic list, to just count those
import json

identities_path =  '../resources/generic_agents-identity_v15_2021_10_15.xlsx'
identities_exclude_path = '../resources/netmapper_exclude.json'
identities_include_path = '../resources/netmapper_include_generic.json'

# Load identity terms
multi_identities = pd.read_excel(identities_path)
with open(identities_exclude_path) as f:
    identities_exclude = json.load(f)
with open(identities_include_path) as f:
    identities_include = json.load(f)

# Filter to English, remove duplicates
cols = multi_identities.columns.tolist()
en_identities = multi_identities[cols[cols.index('English'):]].copy()
en_identities['term'] = en_identities['English'].str.lower()
en_identities.drop_duplicates(subset='term', inplace=True)

# Separate out stopwords
identities = en_identities[
    (en_identities['stop word']!=1) & (~en_identities['term'].isin(identities_exclude))
]
len(identities)

def filter_identities(identities):
    """ Filter identity list to only those present in the data's vocabulary, returns this list.
        Args:
            identities: list of identities to see if are in the vocab
    """

    vocab_path = f'../tmp/incels_vocab.json'

    with open(vocab_path, 'r') as f:
        vocab = json.load(f)

    present_identities = [term for term in identities if term in vocab]
    print(f'\t\t{len(present_identities)} out of {len(identities)} identity terms present in vocab')

    return present_identities

identities = filter_identities(identities['term'].tolist() + identities_include)
len(identities)

		7101 out of 18906 identity terms present in vocab


7101

In [5]:
# Filter matches to just those that are NOT in generic list
incels_novel_exp = incels_exp[~incels_exp['netmapper_identity_matches'].isin(identities)].copy()
print(len(incels_exp))
print(len(incels_novel_exp))

12126310
5754852


In [48]:
# Table for the paper
exclude = ['guy', 'guys', 'gf', 'buddy', 'islam', 'escorts', 'anxiety', 'islamic', 'trannies', 'bruh', 'aryan', 'womens'] # should have been added to the generic list
selected = ([wd for wd in incels_novel_exp.netmapper_identity_matches.value_counts()[:90].index.tolist() if wd not in exclude and 'cel' not in wd])
print(len(selected))
print(', '.join(selected))

46
foids, foid, chads, manlet, stacy, boyo, femoids, ethnics, chadlite, roastie, femoid, stacies, manlets, roasties, holes, betabux, landwhale, deformed, landwhales, waifu, jb, chadlites, chick, chicks, noodlewhores, soyboy, br0, aspie, betas, thots, stacey, brah, jbs, immigration, gook, hoe, qt, waifus, traps, germanic, beckies, crippled, m8, boi, gooks, bud


In [41]:
# Filter matches to just those that are in generic list
incels_generic_exp = incels_exp[incels_exp['netmapper_identity_matches'].isin(identities)].copy()
print(len(incels_exp))
print(len(incels_generic_exp))

12126310
6371458


## Group identity terms

In [ ]:
import json

identity_groups_fpath = '../resources/identity_groups.json'
with open(identity_groups_fpath, 'r') as f:
    identity_groups = json.load(f)
print(len(identity_groups))

incels_exp['identity_group'] = incels_exp.netmapper_identity_matches.map(lambda x: identity_groups.get(x, x))
incels_exploded = incels_exp.explode('identity_group') # Count intersectional mentions as a mention in each of their categories
incels_gp_freq = incels_exploded.identity_group.value_counts().to_frame(name='count').reset_index(names='identity_group')

import plotly.express as px
pd.options.plotting.backend = 'plotly'

fig = px.bar(incels_gp_freq.head(30), x='identity_group', y='count')
fig.update_xaxes(tickangle=45)

### Plot for the paper

In [39]:
selected_gps = [gp for gp in incels_gp_freq.identity_group.head(12).tolist() if gp not in ['men_boys_address', 'women_girls_derogatory', 'incels']]
selected_gps

['women_girls',
 'men_boys',
 'youth',
 'mental_disabilities',
 'lgbtq_people',
 'asian_people',
 'black_people',
 'white_people',
 'jews']

In [45]:
incels_generic_exp['identity_group'] = incels_generic_exp.netmapper_identity_matches.map(lambda x: identity_groups.get(x, x))
incels_generic_exploded = incels_generic_exp.explode('identity_group') # Count intersectional mentions as a mention in each of their categories
incels_generic_gp_freq = incels_generic_exploded.identity_group.value_counts().to_frame(name='count').reset_index(names='identity_group')
incels_generic_gp_freq['freq'] = incels_generic_gp_freq['count']/len(incels_data) # divide by total #posts since dividing by total #mentions doesn't show increase from extending list past generic

ws_exp['identity_group'] = ws_exp.netmapper_identity_matches.map(lambda x: identity_groups.get(x, x))
ws_exploded = ws_exp.explode('identity_group') # Count intersectional mentions as a mention in each of their categories
ws_gp_freq = ws_exploded.identity_group.value_counts().to_frame(name='count').reset_index(names='identity_group')
ws_gp_freq['freq'] = ws_gp_freq['count']/len(ws_data) # divide by total #mentions. Could divide by #posts, bu post length varies between datasets

incels_gp_freq['freq'] = incels_gp_freq['count']/len(incels_data)
gp_freqs = incels_gp_freq[incels_gp_freq['identity_group'].isin(selected_gps)].set_index('identity_group').rename(columns={'freq': 'freq_incels'})
gp_freqs['freq_incels_generic'] = incels_generic_gp_freq.set_index('identity_group')['freq']
gp_freqs['freq_ws'] = ws_gp_freq.set_index('identity_group')['freq']
gp_freqs = gp_freqs.reset_index()
gp_freqs

gp_freqs = pd.wide_to_long(gp_freqs, stubnames='freq', sep='_', i='identity_group', j='dataset', suffix=r'\w+').reset_index().fillna(0)
gp_freqs

,identity_group,dataset,count,freq
0,women_girls,incels,2616242,0.418717
1,men_boys,incels,2034524,0.325616
2,youth,incels,290984,0.046571
3,mental_disabilities,incels,239192,0.038282
4,lgbtq_people,incels,236085,0.037784
5,asian_people,incels,210149,0.033633
6,black_people,incels,156181,0.024996
7,white_people,incels,134838,0.021580
8,jews,incels,103877,0.016625
9,women_girls,incels_generic,2616242,0.290866


In [ ]:
column_map = {'identity_group': 'Identity group', 'dataset': 'Dataset/identity lexicon', 'freq': 'Mentions/post'}
formatted_gp_freqs = gp_freqs.rename(columns=column_map).replace({
    'women_girls': 'Women',
    'men_boys': 'Men',
    'youth': 'Youth',
    'mental_disabilities': 'Neurodiverse',
    'lgbtq_people': 'LGBTQ+',
    'asian_people': 'Asian',
    'black_people': 'Black',
    'white_people': 'White',
    'jews': 'Jews',
    'incels': 'Incels data<br>Expanded lexicon',
    'incels_generic': 'Incels data<br>Generic lexicon',
    'ws': 'White supremacist data<br>Generic lexicon',
})

# fig = px.bar(gp_freqs, x='identity_group', y='freq', color='dataset', barmode='group', title='Identity group mentions per post')
fig = px.bar(formatted_gp_freqs, x=column_map['identity_group'], y=column_map['freq'], color=column_map['dataset'], barmode='group', 
             # title='Identity group mentions per post',
            width=600, height=300)
# fig.update_xaxes(tickangle=45)
fig.update_layout(xaxis_title=None)
fig.write_image('../output/identity_group_mentions.pdf')
fig.write_image('../output/identity_group_mentions.png', scale=3)
fig.show()

In [7]:
from collections import defaultdict

gp_vals = defaultdict(list)
for key, vals in identity_groups.items():
    for gp in vals:
        gp_vals[gp].append(key)
        
for gp, terms in gp_vals.items():
    print(f'{gp}: {terms}')

women_girls: ['women', 'woman', 'girls', 'girl', 'girlfriend', 'girlfriends', 'female', 'females', 'whore', 'whores', 'bitch', 'bitches', 'slut', 'sluts', 'cunt', 'cunts', 'prostitute', 'prostitutes', 'hooker', 'hookers', 'black women', 'feminist', 'feminists', 'white women', 'gf', 'sister', 'mother', 'daughter', 'mom', 'wife', 'wives', 'mothers']
men_boys: ['men', 'man', 'boy', 'boys', 'boyfriend', 'boyfriends', 'bro', 'dude', 'dudes', 'male', 'males', 'black men', 'white men', 'bf', 'brother', 'brothers', 'father', 'son', 'husband', 'husbands', 'fathers']
men_boys_address: ['bro', 'dude', 'dudes']
women_girls_derogatory: ['whore', 'whores', 'bitch', 'bitches', 'slut', 'sluts', 'cunt', 'cunts', 'prostitute', 'prostitutes', 'hooker', 'hookers']
youth: ['kids', 'kid', 'child', 'children', 'baby', 'babies', 'teen', 'teens']
virgins: ['virgin']
lgbtq_people: ['faggot', 'faggots', 'fag', 'fags', 'tranny', 'transgender', 'gay', 'gays', 'trans', 'bi', 'bisexual', 'bisexuals', 'pansexual', 'p

In [8]:
all_identities = exploded.netmapper_identity_matches.unique()
len(all_identities)

5460

In [9]:
for term in gp_vals['latinx_people']:
    print(f'{term}: {term in all_identities}')

spanish: True
hispanic: True
latino: True
latina: False
latinx: False
mexican: True
latin american: False
central american: False
south american: False
dominican: True
puerto rican: False
salvadoran: True
honduran: True
nicaraguan: True
costa rican: False
colombian: True
venezuelan: True
ecuadorian: True
brazilian: True
argentinian: True
bolivian: True
peruvian: True
chilean: True


In [84]:
for term in ['jew', 'jews', 'jewish', 'kike', 'kikes', 'zionist', 'zionists', 'judaism', 'israelite', 'israelites', 'goy', 'goyim']:
    print(f'{term}: {term in all_identities}')

jew: True
jews: True
jewish: False
kike: True
kikes: False
zionist: True
zionists: True
judaism: False
israelite: True
israelites: True
goy: False
goyim: False


In [134]:
for term in ['gay', 'gays', 'lesbian', 'trans', 'transgender', 'bi', 'bisexual', 'pansexual', 'homosexual', 'homo', 'queer',
            'transman', 'transwoman', 'queers', 'fag', 'fags']:
    print(f'{term}: {term in all_identities}')

gay: False
gays: True
lesbian: True
trans: False
transgender: False
bi: False
bisexual: False
pansexual: False
homosexual: True
homo: True
queer: True
transman: False
transwoman: False
queers: False
fag: True
fags: False


In [ ]:
[identity for identity in all_identities if 'negro' in str(identity)]

In [ ]:
[identity for identity in all_identities if 'white' in str(identity)]

In [ ]:
[identity for identity in all_identities if ' ' in str(identity)]

In [ ]:
[identity for identity in all_identities if 'nig' in str(identity)]

In [103]:
import re

re.search(r'\b{}\b'.format(re.escape('black man')), 'black man walking').group()
# re.search(r'\b{}\b'.format(re.escape('black man')), 'asian man walking').group()

'black man'

### Identity groups over time

In [54]:
# For incels

from datetime import datetime, timezone

date_col = 'parsed_date' # for incels data

post_counts = incels_data.groupby(pd.Grouper(key=date_col, freq='MS'))['content'].count()
incels_data['word_count'] = incels_data.content.str.split().str.len() # for incels data

word_counts = incels_data.groupby(pd.Grouper(key=date_col, freq='MS'))['word_count'].sum()
word_counts

# exploded = data.explode('netmapper_identity_matches').dropna(subset='netmapper_identity_matches')
# gp_counts_overtime = exploded.groupby([pd.Grouper(key=date_col, freq='MS'), 'netmapper_identity_matches']).size().to_frame(name='count').reset_index(names=[date_col, 'identity_term'])
gp_counts_overtime = incels_exploded.groupby([pd.Grouper(key=date_col, freq='MS'), 'identity_group']).size().to_frame(name='term_count').reset_index(names=[date_col, 'identity_group'])
gp_counts_overtime

gp_counts_overtime = gp_counts_overtime.join(word_counts, on=date_col).rename(columns={'content': 'word_count'})
gp_counts_overtime

import numpy as np

gp_counts_overtime['mention_prob'] = gp_counts_overtime['term_count'] / gp_counts_overtime.word_count
gp_counts_overtime['log_prob'] = gp_counts_overtime['mention_prob'].map(np.log10)
gp_counts_overtime.head()

,parsed_date,identity_group,term_count,word_count,mention_prob,log_prob
0,2017-11-01,aboriginal,21,3106117,0.000007,-5.169999
1,2017-11-01,aboriginals,24,3106117,0.000008,-5.112007
2,2017-11-01,aborigines,4,3106117,0.000001,-5.890158
3,2017-11-01,abuser,8,3106117,0.000003,-5.589128
4,2017-11-01,abusers,6,3106117,0.000002,-5.714067


In [ ]:
# Plot
import plotly.express as px

overtime_selected_gps = [gp for gp in selected_gps if gp not in ['youth', 'mental_disabilities', 'white_people', 'asian_people']]

# Select only most popular identity groups
# filtered = counts_overtime[counts_overtime.identity_group.isin(gp_freq.identity_group.head(12))]
filtered = gp_counts_overtime[gp_counts_overtime.identity_group.isin(overtime_selected_gps)]

column_map = {'identity_group': 'Identity group', 'parsed_date': 'Date', 'log_prob': 'Log word probability'}
formatted_filtered = filtered.rename(columns=column_map).replace({
    'women_girls': 'Women',
    'men_boys': 'Men',
    'youth': 'Youth',
    'mental_disabilities': 'Neurodiverse',
    'lgbtq_people': 'LGBTQ+',
    'asian_people': 'Asian',
    'black_people': 'Black',
    'white_people': 'White',
    'jews': 'Jews',
    'incels': 'Incels data<br>Expanded lexicon',
    'incels_generic': 'Incels data<br>Generic lexicon',
    'ws': 'White supremacist data<br>Generic lexicon',
})


fig = px.line(formatted_filtered, x=column_map[date_col], y=column_map['log_prob'], color=column_map['identity_group'], width=700, height=400)
# px.line(filtered, x=date_col, y='mention_prob', color='identity_group', log_y=True)
fig.update_layout(xaxis_title=None)
fig.write_image('../output/identity_group_mentions_overtime.pdf')
fig.show()

In [5]:
# For white supremacist data

from datetime import datetime, timezone

# post_counts = data.groupby(pd.Grouper(key=date_col, freq='MS'))['content'].count()
# data['word_count'] = data.content.str.split().str.len() # for incels data

date_col = 'parsed_date' # for incels data
# date_col = 'timestamp' # for white supremacist data, which has a lot of corpus composition changes during that period

# Remove outliers (for white supremacist data)
# data = data[data.timestamp >= datetime(2002, 1, 1, tzinfo=timezone.utc)]
# exploded = exploded[exploded.timestamp >= datetime(2002, 1, 1, tzinfo=timezone.utc)]

word_counts = data.groupby(pd.Grouper(key=date_col, freq='MS'))['word_count'].sum()
word_counts

# exploded = data.explode('netmapper_identity_matches').dropna(subset='netmapper_identity_matches')
# counts_overtime = exploded.groupby([pd.Grouper(key=date_col, freq='MS'), 'netmapper_identity_matches']).size().to_frame(name='count').reset_index(names=[date_col, 'identity_term'])
counts_overtime = exploded.groupby([pd.Grouper(key=date_col, freq='MS'), 'identity_group']).size().to_frame(name='term_count').reset_index(names=[date_col, 'identity_group'])
counts_overtime

counts_overtime = counts_overtime.join(word_counts, on=date_col).rename(columns={'content': 'word_count'})
counts_overtime

import numpy as np

counts_overtime['mention_prob'] = counts_overtime['term_count'] / counts_overtime.word_count
counts_overtime['log_prob'] = counts_overtime['mention_prob'].map(np.log10)
counts_overtime.head()

# Plot
import plotly.express as px

# Select only most popular identity groups
filtered = counts_overtime[counts_overtime.identity_group.isin(gp_freq.identity_group.head(11))]

# px.line(filtered, x=date_col, y='count', color='identity_term')
px.line(filtered, x=date_col, y='log_prob', color='identity_group', height=500)
# px.line(filtered, x=date_col, y='mention_prob', color='identity_group', log_y=True)

KeyError: 'Column not found: word_count'

## Identity categories

In [6]:
import json

identity_cats_fpath = '../resources/identity_categories.json'
with open(identity_cats_fpath, 'r') as f:
    identity_cats = json.load(f)
print(len(identity_cats))

incels_exploded['identity_category'] = incels_exploded.identity_group.map(identity_cats)
cat_freq = incels_exploded.identity_category.value_counts()
cat_freq

433


gender/sexuality    4973903
race/ethnicity       564427
age                  314530
religion             205440
politics              68737
disability             3411
Name: identity_category, dtype: int64

In [ ]:
import plotly.express as px
pd.options.plotting.backend = 'plotly'

fig = px.bar(cat_freq)
fig.show()

In [33]:
from collections import defaultdict

cat_vals = defaultdict(list)
for key, val in identity_cats.items():
    cat_vals[val].append(key)
        
for cat, terms in cat_vals.items():
    print(f'{cat}: {terms}')

race/ethnicity: ['black_people', 'asian_people', 'white_people']
gender/sexuality: ['men_boys', 'women_girls', 'lgbtq_people']
age: ['youth', 'boomers']
disability: ['retard']
religion: ['jews', 'muslims', 'christian', 'hindus', 'buddhists']
politics: ['liberal', 'conservative']


### Identity categories over time

In [ ]:
# post_counts = data.groupby(pd.Grouper(key='parsed_date', freq='MS'))['content'].count()
# data['word_count'] = data.content.str.split().str.len() # already done from identity grouping

word_counts = incels_data.groupby(pd.Grouper(key='parsed_date', freq='MS'))['word_count'].sum()
word_counts

cat_counts_overtime = incels_exploded.groupby([pd.Grouper(key='parsed_date', freq='MS'), 'identity_category']).size().to_frame(name='mention_count').reset_index(names=['parsed_date', 'identity_category'])
cat_counts_overtime

cat_counts_overtime = cat_counts_overtime.join(word_counts, on='parsed_date').rename(columns={'content': 'word_count'})
cat_counts_overtime

import numpy as np

cat_counts_overtime['mention_prob'] = cat_counts_overtime['mention_count'] / cat_counts_overtime.word_count
cat_counts_overtime['log_prob'] = cat_counts_overtime['mention_prob'].map(np.log10)
cat_counts_overtime

# Plot
import plotly.express as px

px.line(cat_counts_overtime, x='parsed_date', y='log_prob', color='identity_category', height=400)

## Plot over time mentions for the paper

In [59]:
cat_counts_overtime

,parsed_date,identity_category,mention_count,word_count,mention_prob,log_prob
0,2017-11-01,age,2845,3106117,0.000916,-3.038136
1,2017-11-01,disability,30,3106117,0.000010,-5.015097
2,2017-11-01,gender/sexuality,68273,3106117,0.021980,-1.657969
3,2017-11-01,politics,537,3106117,0.000173,-3.762244
4,2017-11-01,race/ethnicity,5589,3106117,0.001799,-2.744884
...,...,...,...,...,...,...
247,2021-04-01,disability,10,1321763,0.000008,-5.121154
248,2021-04-01,gender/sexuality,26992,1321763,0.020421,-1.689919
249,2021-04-01,politics,691,1321763,0.000523,-3.281676
250,2021-04-01,race/ethnicity,6480,1321763,0.004903,-2.309579


In [60]:
cat_counts_overtime['identity_group'] = cat_counts_overtime['identity_category']
selected_cat_counts_overtime = cat_counts_overtime[cat_counts_overtime['identity_group']=='politics']

In [ ]:
# Plot
import plotly.express as px

overtime_selected_gps = [gp for gp in selected_gps if gp not in ['youth', 'mental_disabilities', 'white_people', 'asian_people']]

# Select only most popular identity groups
# filtered = counts_overtime[counts_overtime.identity_group.isin(gp_freq.identity_group.head(12))]
filtered = pd.concat([gp_counts_overtime[gp_counts_overtime.identity_group.isin(overtime_selected_gps)], selected_cat_counts_overtime])

column_map = {'identity_group': 'Identity group/category', 'parsed_date': 'Date', 'log_prob': 'Log word probability'}
formatted_filtered = filtered.rename(columns=column_map).replace({
    'women_girls': 'Women',
    'men_boys': 'Men',
    'youth': 'Youth',
    'mental_disabilities': 'Neurodiverse',
    'lgbtq_people': 'LGBTQ+',
    'asian_people': 'Asian',
    'black_people': 'Black',
    'white_people': 'White',
    'jews': 'Jews',
    'incels': 'Incels data<br>Expanded lexicon',
    'incels_generic': 'Incels data<br>Generic lexicon',
    'ws': 'White supremacist data<br>Generic lexicon',
    'politics': 'Politics'
})


fig = px.line(formatted_filtered, x=column_map[date_col], y=column_map['log_prob'], color=column_map['identity_group'], width=800, height=400)
# px.line(filtered, x=date_col, y='mention_prob', color='identity_group', log_y=True)
fig.update_layout(xaxis_title=None)
fig.write_image('../output/identity_group_mentions_overtime.pdf')
fig.write_image('../output/identity_group_mentions_overtime.png', scale=3)
fig.show()

## Look into specific time periods

In [18]:
# Filter data to that month

# April 2020 jump in women_girls_derogatory and black_people
# period = incels_exploded[(exploded.parsed_date.dt.year == 2020) & (exploded.parsed_date.dt.month == 4)]

# Aug 2019 peak before mentions of Jews drop
# period = incels_exploded[(incels_exploded.parsed_date.dt.year == 2019) & (incels_exploded.parsed_date.dt.month == 8)]

# Nov 2019 mentions of Jews drop
period = incels_exploded[(incels_exploded.parsed_date.dt.year == 2019) & (incels_exploded.parsed_date.dt.month == 10)]
period.info()
print(period.parsed_date.min())
print(period.parsed_date.max())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327498 entries, 914 to 6247220
Data columns (total 13 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   type                              327498 non-null  object             
 1   forum                             327498 non-null  object             
 2   thread                            327498 non-null  object             
 3   username                          327498 non-null  object             
 4   date                              327498 non-null  object             
 5   content                           327498 non-null  object             
 6   parsed_date                       327498 non-null  datetime64[ns, UTC]
 7   content_orig                      325520 non-null  object             
 8   netmapper_identity_matches        266129 non-null  object             
 9   netmapper_identity_matches_spans  327498 non-

In [ ]:
# identity_group = 'black_people'
identity_group = 'jews'
# identity_group = 'women_girls_derogatory'
selected = period[period.identity_group == identity_group]
print(len(selected))
print(len(selected.index.unique())) # index is unique to the post

# selected.index.value_counts().iloc[:20] # posts with most mentions of that group
selected[['content', 'parsed_date']].sample(10, random_state=9)

In [11]:
pd.set_option('display.max_colwidth', None)
selected.loc[1783573].iloc[0]

type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
pd.set_option('display.max_colwidth', None)
selected.loc[1045225].iloc[0]

# Check particular regex matches

In [8]:
import re

pat = r'\bincels(?!\.co|\.is)\b'
# test_text = 'incels'
# test_text = 'incels.is'
# test_text = 'incels.co is the best'
test_text = 'incels is the best'

re.search(pat, test_text)

<re.Match object; span=(0, 6), match='incels'>

In [9]:
re.escape(pat)

'\\\\bincels\\(\\?!\\\\\\.co\\|\\\\\\.is\\)\\\\b'

# Old/1-time

In [18]:
# Count number of identity term mentions (top-down generic list + bottom-up)
print(exp['netmapper_identity_matches'].count())

8909464


In [4]:
# Count number of identity term mentions (top-down generic list)
print(exp['netmapper_identity_matches'].count())

6464047

In [96]:
# Filter out capitalized terms (unnecessary now)
identity_cats = {term: cat for term, cat in identity_cats.items() if term[0].islower()}
len(identity_cats)

723

In [3]:
# Load data with identity matches
import pandas as pd

path = '../../data/incels/processed_comments.jsonl'
data = pd.read_json(path, lines=True, convert_dates=['parsed_date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248230 entries, 0 to 6248229
Data columns (total 8 columns):
 #   Column                                 Dtype         
---  ------                                 -----         
 0   type                                   object        
 1   forum                                  object        
 2   thread                                 object        
 3   username                               object        
 4   date                                   object        
 5   content                                object        
 6   parsed_date                            datetime64[ns]
 7   {self.identity_list}_identity_matches  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 381.4+ MB


In [4]:
data.rename(columns={'{self.identity_list}_identity_matches': 'netmapper_identity_matches'}, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248230 entries, 0 to 6248229
Data columns (total 8 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   type                        object        
 1   forum                       object        
 2   thread                      object        
 3   username                    object        
 4   date                        object        
 5   content                     object        
 6   parsed_date                 datetime64[ns]
 7   netmapper_identity_matches  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 381.4+ MB


In [5]:
data.to_pickle('../../data/incels/processed_comments.pkl')